# 예제 3.35-3.43: 비선형 회귀

## 학습목표
1. **커스텀 데이터셋(Custom Dataset)** 클래스 작성법 익히기
2. **커스텀 모델(Custom Model)** 클래스 작성법 익히기
3. **nn.Module 상속**을 통한 모델 정의 방법 학습하기
4. **비선형 데이터**를 선형 모델로 학습하는 기법 이해하기
5. **모델 저장** 방법 학습하기

---

#### 예제 3.35 라이브러리 임포트

In [ ]:
import torch
import pandas as pd
from torch import nn
from torch import optim
# Dataset: 커스텀 데이터셋 클래스의 부모 클래스
from torch.utils.data import Dataset, DataLoader

---

#### 예제 3.36-3.38 커스텀 데이터셋 클래스

**Dataset 클래스 필수 메서드**
- `__init__`: 데이터 로드 및 초기화
- `__getitem__`: 인덱스로 샘플 반환
- `__len__`: 데이터셋 크기 반환

**비선형 → 선형 변환**: [x², x] 형태로 특성 생성

In [ ]:
class CustomDataset(Dataset):
    """커스텀 데이터셋 클래스"""
    
    def __init__(self, file_path):
        """CSV 파일에서 데이터 로드"""
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values  # 첫 번째 열: 입력
        self.y = df.iloc[:, 1].values  # 두 번째 열: 타겟
        self.length = len(df)

    def __getitem__(self, index):
        """인덱스로 샘플 반환 - 비선형을 선형으로 변환"""
        # y = ax² + bx 형태를 [x², x]로 변환하여 선형 학습 가능
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        """데이터셋 크기 반환"""
        return self.length

---

#### 예제 3.39-3.40 커스텀 모델 클래스

**nn.Module 상속**
- `__init__`: 레이어 정의
- `forward`: 순전파 로직 정의

In [ ]:
class CustomModel(nn.Module):
    """커스텀 모델 클래스 (nn.Module 상속)"""
    
    def __init__(self):
        """레이어 초기화"""
        super().__init__()  # 부모 클래스 초기화 필수
        self.layer = nn.Linear(2, 1)  # 입력 2 → 출력 1

    def forward(self, x):
        """순전파 정의"""
        x = self.layer(x)
        return x

---

#### 예제 3.41 데이터로더 생성

In [ ]:
# 커스텀 데이터셋 인스턴스 생성
train_dataset = CustomDataset("../datasets/non_linear.csv")

# 데이터로더: batch_size=128로 대용량 데이터 처리
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

---

#### 모델, 손실함수, 옵티마이저 정의

**GPU 사용**: `.to(device)`로 모델과 손실함수를 GPU로 이동

In [ ]:
# GPU 사용 가능 여부 확인
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델을 GPU로 이동
model = CustomModel().to(device)

# 손실함수도 GPU로 이동
criterion = nn.MSELoss().to(device)

# 옵티마이저 설정
optimizer = optim.SGD(model.parameters(), lr=0.0001)

---

#### 예제 3.42 학습 루프

In [ ]:
# 학습 루프
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        # 데이터를 GPU로 이동
        x = x.to(device)
        y = y.to(device)

        output = model(x)          # 순전파
        loss = criterion(output, y)  # 손실 계산

        optimizer.zero_grad()  # 기울기 초기화
        loss.backward()        # 역전파
        optimizer.step()       # 파라미터 업데이트

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

---

#### 예제 3.43 모델 추론

**추론 모드**
- `torch.no_grad()`: 기울기 계산 비활성화 (메모리 절약)
- `model.eval()`: 평가 모드로 전환

In [ ]:
# 추론 (예측) 수행
with torch.no_grad():  # 기울기 계산 비활성화
    model.eval()  # 평가 모드로 전환
    
    # 테스트 입력: [x², x] 형태로 변환
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],    # x=1일 때
            [5 ** 2, 5],    # x=5일 때
            [11 ** 2, 11]   # x=11일 때
        ]
    ).to(device)
    
    outputs = model(inputs)
    print(outputs)  # 예측 결과 출력

---

#### 모델 저장

**두 가지 저장 방식**
1. `torch.save(model, path)`: 모델 전체 저장
2. `torch.save(model.state_dict(), path)`: 파라미터만 저장 (권장)

In [ ]:
# 방법 1: 모델 전체 저장 (구조 + 파라미터)
torch.save(
    model,
    "../models/model.pt"
)

In [ ]:
# 방법 2: 모델 상태(파라미터)만 저장 (권장)
torch.save(
    model.state_dict(),
    "../models/model_state_dict.pt"
)